# **Test de grabación y transcripción de audio**

Este notebook usa las funciones de `utils_audio.py` para:
1. Grabar audio desde el micrófono (usando `sounddevice`).
2. Preprocesar el audio.
3. Transcribir con Whisper.

Si la grabación no es posible en el entorno, cargará archivos `.wav` de la carpeta `audio/`.

---
### **Librerías**

In [1]:
# Importar utilidades de audio
from utils_audio import load_whisper_model, preprocess_audio_file, transcribe_audio_file
import sounddevice as sd
import numpy as np
import tempfile, os
from scipy.io.wavfile import write
from IPython.display import Audio
import soundfile as sf
import torch
import librosa

# Ignorar los warnings 
import warnings
warnings.filterwarnings("ignore")

---
### **1. Grabar audio desde el micrófono**

In [2]:
# Parámetros de grabación
fs = 16000  # frecuencia de muestreo
duration = 5  # segundos
print(f'Grabando {duration}s de audio a {fs}Hz...')
# Record
try:
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Esperar a que termine
    print('Grabación completada')
    # Guardar a WAV temporal
    tmp_wav = os.path.join(tempfile.gettempdir(), 'test_record.wav')
    write(tmp_wav, fs, np.squeeze(recording))
    display(Audio(tmp_wav, rate=fs))
    audio_path = tmp_wav
except Exception as e:
    
    print('No fue posible grabar:', e)
    # Fallback: tomar primer archivo de carpeta 'audio'
    files = [f for f in os.listdir('audio') if f.lower().endswith('.wav')]
    if files:
        audio_path = os.path.join('audio', files[0])
        print('Usando archivo de prueba:', audio_path)
        display(Audio(audio_path))
    else:
        raise RuntimeError('No hay forma de obtener audio')


Grabando 5s de audio a 16000Hz...
Grabación completada


---
### **2. Preprocesar y transcribir**

In [3]:
# Modelos disponibles en Whisper 
available_models = ["tiny", "base", "small", "medium", "large"]

# Pedir al ususario el modelo a usar
#model_size = input(f"¿Qué modelo usar? ({', '.join(available_models)}): ").strip().lower()
model_size = "medium" # mejor este de momento

In [4]:
# Cargar modelo Whisper (si no está ya cargado)
model, device = load_whisper_model(model_size)

# Preprocesar archivo
proc_path = preprocess_audio_file(audio_path, target_sr=16000)
print('Archivo preprocesado en:', proc_path)

# Función para transcribir un array NumPy con Whisper
def transcribe_numpy(audio_array: np.ndarray, sr: int):
    # Asegurar que el dtype es float32
    audio_array = audio_array.astype(np.float32)
    # Convertir a tensor y mover al dispositivo
    audio_tensor = torch.from_numpy(audio_array).to(device)
    # Ejecutar la transcripción
    result = model.transcribe(audio_tensor)
    return result.get('text', '').strip()

# Leer datos PCM directamente
audio_array, sr = sf.read(proc_path)
print(f"Leídos {audio_array.shape[0]} muestras a {sr} Hz")

# Re-muestreo si hace falta
if sr != 16000:
    import librosa
    audio_array = librosa.resample(audio_array, orig_sr=sr, target_sr=16000)
    print(f"Re-muestreado a {audio_array.shape[0]} muestras a 16000 Hz")

# Ejecutar transcripción
text = transcribe_numpy(audio_array, sr)
print('Transcripción:')
print(text)


Archivo preprocesado en: /var/folders/z2/b_41nsk11996m00f1c0095th0000gn/T/test_record_16k_mono.wav
Leídos 80000 muestras a 16000 Hz
Transcripción:
Hola, ¿qué tal estás? Hola, hola, hola, hola, esto es una prueba, hola.
